In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
data = pd.read_csv('Food.csv')

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        217 non-null    int64  
 1   Food      217 non-null    object 
 2   Grams     217 non-null    object 
 3   Calories  217 non-null    float64
 4   Protein   217 non-null    object 
 5   Category  217 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 10.3+ KB


In [12]:
data['Grams'] = data['Grams'].str.replace(',', '.').astype(float)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        217 non-null    int64  
 1   Food      217 non-null    object 
 2   Grams     217 non-null    float64
 3   Calories  217 non-null    float64
 4   Protein   217 non-null    object 
 5   Category  217 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 10.3+ KB


In [14]:
data['Protein'] = data['Protein'].str.replace('t', '0').astype(float)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        217 non-null    int64  
 1   Food      217 non-null    object 
 2   Grams     217 non-null    float64
 3   Calories  217 non-null    float64
 4   Protein   217 non-null    float64
 5   Category  217 non-null    object 
dtypes: float64(3), int64(1), object(2)
memory usage: 10.3+ KB


### Rekomendasi


In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        217 non-null    int64  
 1   Food      217 non-null    object 
 2   Grams     217 non-null    float64
 3   Calories  217 non-null    float64
 4   Protein   217 non-null    float64
 5   Category  217 non-null    object 
dtypes: float64(3), int64(1), object(2)
memory usage: 10.3+ KB


In [32]:
#convert categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_features=encoder.fit_transform(data[['Category']])

In [33]:
#combine features
feature = data[['Calories']]
features = pd.concat([feature, pd.DataFrame(encoded_features)],axis=1)

In [34]:
#hitung cosine similarity
cosine_sim= cosine_similarity(features,features)

In [39]:
#fungsi to get top N food recommendation based od give feature
def model_food(query_Categori,query_Calories,top_n=10):
    query_encoded = encoder.transform([[query_Categori]])
    query_feature = [query_Calories] + list(query_encoded[0])

    similarity_scores = cosine_similarity([query_feature], features)
    similarity_food_indices = similarity_scores[0].argsort()[-top_n:][::1]
    similarity_food_info =data.iloc[similarity_food_indices]
    return similarity_food_info

# query_Protein = 20
query_Categori = 'Fish, Seafood'
query_Calories = 200
recommendations = model_food(query_Categori, query_Calories)
print('rekomendasi makanan :')
print(recommendations[['Food','Grams','Calories','Protein','Category']])

rekomendasi makanan :
                 Food  Grams  Calories  Protein       Category
82               Shad   85.0     170.0     20.0  Fish, Seafood
68                Cod  100.0     170.0     28.0  Fish, Seafood
85               Tuna   85.0     170.0     25.0  Fish, Seafood
77            Oysters  230.0     231.0    232.0  Fish, Seafood
80           Sardines   85.0     180.0     22.0  Fish, Seafood
84          Swordfish  100.0     180.0     27.0  Fish, Seafood
73            Halibut  100.0     182.0     26.0  Fish, Seafood
74            Herring  100.0     211.0     22.0  Fish, Seafood
71           Flounder  100.0     200.0     30.0  Fish, Seafood
70  Fish sticks fried  112.0     200.0     19.0  Fish, Seafood


C:\Users\bahbah\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [41]:
# Save the encoder and recommendation function to a pickled file
with open('recommendation_model.pkl', 'wb') as f:
    pickle.dump((encoder, model_food), f)